In [1]:
%matplotlib inline

import numpy as np
import pyvista as pv
from scipy.spatial import KDTree
from binary_to_csv import binary_to_csv
import os
import pandas as pd

In [2]:
DATA_DIR = '../data'
BIN_FILE_NAME = "data_20241015_183835.bin"

BIN_FILE_PATH = os.path.join(DATA_DIR, BIN_FILE_NAME)
CSV_FILE_PATH = os.path.join(DATA_DIR, BIN_FILE_NAME.replace(".bin", ".csv"))


binary_to_csv(BIN_FILE_PATH, CSV_FILE_PATH)
print(CSV_FILE_PATH)

df = pd.read_csv(CSV_FILE_PATH, header=None)
df.head()

../data/data_20241015_183835.csv


,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
0,12026,12146,12195,7700,11000,6650,3713,8060,11381,8158,...,5595,4949,5279,5441,1395,5540,4835,6138,4396,2115
1,12033,12162,12190,7709,10998,6661,3707,8060,11378,8164,...,5582,4956,5263,5443,1390,5546,4832,6141,4390,2116
2,12028,12158,12203,7699,10999,6658,3723,8054,11376,8165,...,5579,4947,5271,5447,1374,5546,4830,6139,4382,2115
3,12026,12162,12197,7709,10992,6655,3712,8061,11374,8161,...,5586,4956,5279,5450,1385,5545,4831,6136,4385,2114
4,12034,12171,12202,7693,11004,6667,3723,8044,11383,8172,...,5577,4931,5264,5448,1381,5537,4827,6135,4388,2111


In [3]:
ASSETS_DIR = '../assets'

grid_points = np.load(os.path.join(ASSETS_DIR, "boob_grid_12_21.npy"))

# Remove points that are outside of the boob mesh.
# Because those points have been skipped during the points mapping, their value is (0, 0, 0)
origin = np.array([0, 0, 0])
points_to_skip = np.where((grid_points == origin).all(axis=1))[0]
points_of_interest = np.delete(grid_points, points_to_skip, axis=0)

# Drop the columns that represent points out of the breast model.
df_poi = df.drop(columns=points_to_skip)

df_poi.head()

,2,3,4,5,14,15,16,17,18,19,...,238,242,243,244,245,246,247,248,249,250
0,12195,7700,11000,6650,12273,12036,11223,11211,60712,7879,...,4411,5595,4949,5279,5441,1395,5540,4835,6138,4396
1,12190,7709,10998,6661,12276,12045,11223,11214,60711,7887,...,4404,5582,4956,5263,5443,1390,5546,4832,6141,4390
2,12203,7699,10999,6658,12278,12041,11213,11215,60717,7889,...,4397,5579,4947,5271,5447,1374,5546,4830,6139,4382
3,12197,7709,10992,6655,12267,12045,11217,11204,60708,7869,...,4405,5586,4956,5279,5450,1385,5545,4831,6136,4385
4,12202,7693,11004,6667,12272,12019,11217,11216,60722,7878,...,4406,5577,4931,5264,5448,1381,5537,4827,6135,4388


In [4]:
# Create the point cloud and initialize the scalars
point_cloud = pv.PolyData(points_of_interest)

p1 = pv.Plotter(border=False)
p1.add_title("Breast points", font_size=12)
p1.add_mesh(point_cloud, copy_mesh=True ,scalars=np.zeros(points_of_interest.shape[0]), cmap='cool', point_size=15)

p1.remove_scalar_bar()

p1.camera_position = 'xy'
p1.camera.elevation += 100

p1.show()


Widget(value='<iframe src="http://localhost:58731/index.html?ui=P_0x302039850_0&reconnect=auto" class="pyvista…

Error handling request
Traceback (most recent call last):
  File "/Users/severinferard/Code/boob_pressure_map/.venv/lib/python3.12/site-packages/aiohttp/web_protocol.py", line 456, in _handle_request
    resp = await request_handler(request)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/severinferard/Code/boob_pressure_map/.venv/lib/python3.12/site-packages/aiohttp/web_app.py", line 537, in _handle
    resp = await handler(request)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/severinferard/Code/boob_pressure_map/.venv/lib/python3.12/site-packages/wslink/backends/aiohttp/__init__.py", line 234, in handleWsRequest
    await self.onMessage(is_binary(msg), msg, client_id)
  File "/Users/severinferard/Code/boob_pressure_map/.venv/lib/python3.12/site-packages/wslink/protocol.py", line 260, in onMessage
    await self.onCompleteMessage(full_message, client_id)
  File "/Users/severinferard/Code/boob_pressure_map/.venv/lib/python3.12/site-packages/wslink/protocol.py", line 280, i

In [5]:
K = 5

# Load the boob 3D model from file
boob_mesh = pv.read( '../assets/boob.obj')
boob_mesh.translate(np.array([0, -0.035, 0]), inplace=True)

# Create a kd-tree for quick nearest-neighbor lookup.
kdtree = KDTree(points_of_interest)

# Find the K nearest point_cloud points for each points in the boob mesh and calculate their respective distances
boob_mesh['nearest_points'] = kdtree.query(boob_mesh.points, k=K)[1]
boob_mesh['nearest_points_dist'] = kdtree.query(boob_mesh.points, k=K)[0]

# Step 1: Calculate the reciprocal of the distances
reciprocal_distances = 1 / boob_mesh['nearest_points_dist']

# Step 2: Square the reciprocal distances
squared_reciprocal_distances = reciprocal_distances ** 1.2


# Step 3: Normalize the weights
normalized_weights = squared_reciprocal_distances / \
    np.sum(squared_reciprocal_distances, axis=1, keepdims=True)

boob_mesh['pressure'] = np.zeros(boob_mesh.points.shape[0])
boob_mesh['max_pressure'] = boob_mesh['pressure']


p4 = pv.Plotter(border=False)
p4.add_title("Breast model", font_size=12)
p4.add_mesh(boob_mesh, copy_mesh=True , scalars=np.zeros(boob_mesh.points.shape[0]), cmap='cool', point_size=15)

p4.remove_scalar_bar()

p4.camera_position = 'xy'
p4.camera.elevation += 100

p4.show()



Widget(value='<iframe src="http://localhost:58731/index.html?ui=P_0x308bd8f80_1&reconnect=auto" class="pyvista…

In [6]:
calibration_data = df_poi.iloc[:20].median()

In [7]:
def interpolate_grid_on_model(grid_scalar):
    return np.sum(grid_scalar[boob_mesh['nearest_points']] * normalized_weights, axis=1)

In [11]:
THRESHOLD = 100

p2 = pv.Plotter( border=False, shape=(1, 2))

points_touched = (df_poi > calibration_data + THRESHOLD).any().values
point_cloud['pressed'] = points_touched

p2.subplot(0, 0)
p2.add_title("Points pressed", font_size=8)
p2.add_mesh(point_cloud, copy_mesh=True, scalars='pressed', cmap='cool', point_size=10)


s = np.sum(points_touched[boob_mesh['nearest_points']] * normalized_weights, axis=1)


p2.subplot(0, 1)
p2.add_title("Points pressed", font_size=8)
p2.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(points_touched), cmap='cool', point_size=10)

p2.link_views()

p2.camera_position = 'xy'
p2.camera.elevation += 100



p2.show()

Widget(value='<iframe src="http://localhost:58731/index.html?ui=P_0x308c9bbc0_5&reconnect=auto" class="pyvista…

In [10]:

relative_pressure = df_poi - calibration_data

p3 = pv.Plotter(border=False, shape=(2, 2))

p3.subplot(0, 0)
p3.add_title("Mean Pressure", font_size=8)
p3.add_mesh(point_cloud, copy_mesh=True, scalars=relative_pressure.mean(), scalar_bar_args={
             'title': 'mean', "title_font_size": -1}, cmap='cool', point_size=10)

p3.subplot(1, 0)
p3.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(relative_pressure.mean().values), scalar_bar_args={
             'title': 'mean', "title_font_size": -1}, cmap='cool', point_size=10)

p3.subplot(0, 1)
p3.add_title("Max Pressure", font_size=8)
p3.add_mesh(point_cloud, copy_mesh=True, scalars=relative_pressure.max(), scalar_bar_args={
             'title': 'max', "title_font_size": -1}, cmap='cool', point_size=10)

p3.subplot(1, 1)
p3.add_mesh(boob_mesh, copy_mesh=True, scalars=interpolate_grid_on_model(relative_pressure.max().values), scalar_bar_args={
             'title': 'max', "title_font_size": -1}, cmap='cool', point_size=10)

p3.link_views()


p3.camera_position = 'xy'
p3.camera.elevation += 100

p3.show(jupyter_backend='trame')



Widget(value='<iframe src="http://localhost:58731/index.html?ui=P_0x308c9b560_4&reconnect=auto" class="pyvista…